In [2]:
import os
%pwd

'd:\\Deep Learning Projects\\Deep-learning-project-mlflow-dvc\\Deep-Learning-Project-MLFlow-DVC\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Deep Learning Projects\\Deep-learning-project-mlflow-dvc\\Deep-Learning-Project-MLFlow-DVC'

In [4]:
import dagshub
dagshub.init(repo_owner='ishita-jainn', repo_name='Deep-Learning-Project-MLFlow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-12-08 23:08:41,655: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as ishita-jainn

[2025-12-08 23:08:41,702: INFO: helpers: Accessing as ishita-jainn]
[2025-12-08 23:08:42,525: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/ishita-jainn/Deep-Learning-Project-MLFlow-DVC "HTTP/1.1 200 OK"]
[2025-12-08 23:08:43,287: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "ishita-jainn/Deep-Learning-Project-MLFlow-DVC"

[2025-12-08 23:08:43,291: INFO: helpers: Initialized MLflow to track repo "ishita-jainn/Deep-Learning-Project-MLFlow-DVC"]


Repository ishita-jainn/Deep-Learning-Project-MLFlow-DVC initialized!

[2025-12-08 23:08:43,294: INFO: helpers: Repository ishita-jainn/Deep-Learning-Project-MLFlow-DVC initialized!]


In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [17]:
 class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri = "https://dagshub.com/ishita-jainn/Deep-Learning-Project-MLFlow-DVC.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [21]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod #can access from anywhere. independent function. wont be needing self.
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model , "model", registered_model_name = "VGG16Model")
            
            else:
                mlflow.keras.log_model(self.model, "model")


In [20]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()


except Exception as e:
    raise e

[2025-12-09 00:02:56,186: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-12-09 00:02:56,194: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-12-09 00:02:56,198: INFO: common: created directory at : artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 42s 4s/step - loss: 4.3915 - accuracy: 0.5036


2025/12/09 00:03:42 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-12-09 00:03:45,545: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2025-12-09 00:03:47,065: INFO: builder_impl: Assets written to: C:\Users\ISHITA\AppData\Local\Temp\tmpcbrbs5y4\model\data\model\assets]


c:\Users\ISHITA\anaconda3\envs\cnn-new\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/12/09 00:04:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
